In [1]:
function calculate_columns(node)
    node_pool=Array{Array{Int,1},1}()

    push!(node_pool,vcat(10000,ones(Int,P+1)))
    for k in 2:size(column_pool,1) # parcourt tous les patterns
        add=true

        # On elimine les colonnes sous 2 conditions
        # 1. Si x_ji <=0 et que l'arc (j,i) existe
        for (i,j) in tree[node].setzero
            if (column_pool[k][i+1]==1)
                add=false
            end
        end
        
        # 2. Si x_ji >=1 et que l'arc (j,i) n'existe pas
        for (i,j) in tree[node].setone
            if (column_pool[k][i+1]==0)
                add=false
            end
        end

        if add 
            push!(node_pool,column_pool[k])
        end
    end

    return node_pool
end

calculate_columns (generic function with 1 method)

In [ ]:
# Transformer la solution α en x
function calculate_xsol(mastersol,node_pool)
    x=Array{Array{Float32},1}(undef,P)
    for i in 1:P
        x[i]=Array{Float32,1}(undef,P)
    end

    for i in 1:P
        for j in 1:P
            x[i][j]=0
            for k in 1:size(node_pool,1)
                x[i][j]=mastersol[k]*node_pool[k][j+1]+x[i][j]
            end
        end
    end
    print("x",x)
    print("\n")
    return x
end

In [ ]:
function Process_Node(nodeindex)
    println("\e[93m Noeud traité : $nodeindex \e[00m")
    println("Variables mises à 0 sont :", tree[nodeindex].setzero)
    println("Variables mises à 1 sont :", tree[nodeindex].setone)
    global nodestobedeleted = []
    
    # Calcul des colonnes compatibles avec les contraintes de branchements
    node_pool = calculate_columns(nodeindex)
    
    # On prend le premier maitre restreint
    paire,z,mastermodel=node_master(node_pool)
    
    while true
        # Résolution du maitre restreint
        optimize!(mastermodel)
        
        node_infeasible = false
        
        # Stockage de la solution duale 
        π = JuMP.dual.(paire)
        
        # Stockage de la valeur objective du problème maitre actuel
        value = JuMP.objective_value(mastermodel)
    
        # Stockage de la solution
        #solution=Array{Array{Float32,1},1}(undef,P)
        #for i in 1:P
        #    solution[i]=Array{Float32,1}(undef,size(node_pool,1))
        #    for j in 1:size(node_pool,1)
        #        solution[i][j]=JuMP.value.(z[i])[j]
        #    end
        #end
        
        solution=Array{Float32,1}(undef,P)
        for i in 1:size(node_pool,1)
            solution[i]=JuMP.value.(z[i])
        end
        solflat = collect(Iterators.flatten(solution))
      
        # Vérification si la solution est entière
        if maximum(solflat-floor.(solflat)) <= ϵ
            if value <= UB
                global UB=value
            end
            println("\e[32m Solution admissible avec valeur $value trouvée \e[00m")
            
            for i in 1:length(Queue)-1
                if tree[Queue[i]].lb >= UB
                    push!(nodestobedeleted,i)
                end
            end
        end
 
        # Mise a jour des bornes supérieure et inférieure au noeud actuel
        nodeub = value
        nodelb = sum(subproblem(π,nodeindex)[1])
        minobj = 0
        
        
        # Résolution du sous-problème
        SPobj,column = subproblem(π,nodeindex)

        if SPobj<Inf 
            # Extraction du motif de la solution
            pattern=findall(x->x!=0,column)
            # Calcul du cout réduit du motif
            reduced_cost = SPobj

            if reduced_cost<=minobj 
                minobj=reduced_cost
            end

            # Si un motif à cout réduit est trouvé, on l'ajoute aux patterns
            if reduced_cost<0
                # Calcul du cout de la colonne qui sera une contribution à la fonction objective
                cout_colonne=sum(w[i,j]*column(j) for i in 1:P,j in 1:P)
                # Ajout colonne dans column_pool
                push!(column_pool,vcat(cout_colonne,vcat(column,1)))
                # Ajout colonne dans node_pool
                push!(node_pool,vcat(cout_colonne,vcat(column,1)))
                
                # Ajout de la colonne au modèle
                # Ajout de la nouvelle variable au vecteur des variables
                push!(z,@variable(mastermodel,lower_bound=0))
                # Mis à jour du nom
                set_name(z[end], "z_$(size(column_pool,1))")
                # Donner les coefficients aux variables dans les contraintes liées aux paires
                for j in 1:P
                    set_normalized_coefficient(paire[j], z[end], column[j])
                end
                
                # Mise a jour de la fonction objective
                set_objective_function(mastermodel, objective_function(mastermodel) + column_cost*z[end])
                
                println("\e[34m Pattern avec articles $pattern ajouté")
            end
            # Calcul de la nouvelle borne inférieure
            nodelb += SPobj
        else
            node_infeasible = true
            println("\e[31m Noeud infaisable \e[00m")
            break
        end
 
        
        # Si le sous-problème n'est pas faisable alors le noeud n'est pas faisable
        if node_infeasible
            return []
        end
      
        # Si la borne inférieure est améliorée, alors on la met à jour
        if nodelb>=tree[nodeindex].lb
            tree[nodeindex].lb=nodelb
        end
        
        # Si il y a convergence, on s'arrête
        if 2*abs((nodeub-tree[nodeindex].lb))/abs((nodeub+tree[nodeindex].lb))<ϵ || minobj>=0
            println("Relaxation au noeud est résolue à l'optimalité")
            println("La borne supérieure au noeud est $nodeub")
            println("La borne inférieure au noeud est $(tree[nodeindex].lb)")
    
            # Si la colonne artificielle est utilisée dans la solution finale alors le maitre est infaisable
            if solution[1]>=ϵ
                node_infeasible = true
                println("\e[31m Noeud est infaisable \e[00m")
            end
  
            if node_infeasible
                return []
            else
                return calculate_xsol(solution,node_pool)
            end
        end
    end
end